# Unsupervised Learning : Assignment 2

TEAM Details:

<b>Group ID</b> - 13 <br>

<b>Group Name</b> - Data Engineers<br>

<b>Group Members</b>:<br>
2020AIML030 - Karunya DS <br>
2020AIML069 - Samtani Edwin<br>
2020AIML022 - Apoorva ventra<br>

## Assignment

Task in this assignment is to:

1. Do appropriate EDA and preprocessing on the data assigned to your group

2. Cluster the data using the algorithms and details mentioned below as a guideline and find out which works best for your data

3. After finding the best clustering obtained for your data, try to interpret some/all of the clusters and derive some domain-specific insights out of the results

Clustering starter ideas:

1. You can check Agglomerative clustering for different linkages (Ward, Complete, Average) and different distance measures (Euclidean, Manhattan, Cosine)

2. You can check DBSCAN clustering for different min_pts and eps values.

Use any clustering metric of your choice to track the performance of the different algorithms and select the final clustering for interpretation.

 

Note: Do not limit your submission notebook to the final algorithm that worked for you, include all experiments, EDA, plots etc in the submission.

## Import required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
jewellery_df = pd.read_csv('jewellery.csv')
jewellery_df.head()

,Age,Income,SpendingScore,Savings
0,58,77769,0.791329,6559.829923
1,59,81799,0.791082,5417.661426
2,62,74751,0.702657,9258.992965
3,59,74373,0.765680,7346.334504
4,87,17760,0.348778,16869.507130
